In [1]:
import io
import os
import shutil
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
import math
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_selector, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, f1_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR, SVC
from sklearn.neural_network import MLPRegressor
from sklearn.inspection import permutation_importance

In [2]:
conjuntos = ['treinamento', 'teste', 'validacao']
base_path = 'dados/processed'
pastas_excluir = ['cleanImage', '?']

for conjunto in conjuntos:
    conjunto_path = os.path.join(base_path, conjunto)
    for nome in os.listdir(conjunto_path):
        full_path = os.path.join(conjunto_path, nome)
        if nome in pastas_excluir and os.path.isdir(full_path):
            print(f"Removendo pasta: {full_path}")
            shutil.rmtree(full_path)

In [3]:
train_dir = 'dados/processed/treinamento'
val_dir   = 'dados/processed/validacao'
test_dir  = 'dados/processed/teste'

In [4]:
def load_images_from_folder(folder):
    images = {}
    for char_folder in os.listdir(folder):
        char_images = []
        for filename in os.listdir(os.path.join(folder, char_folder)):
            if filename.endswith('.jpg'):
                img_path = os.path.join(folder, char_folder, filename)
                if os.path.isfile(img_path):
                    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                    if img is not None:
                        char_images.append(img)
        
        images[char_folder] = char_images
    
    return images

In [5]:
def calculate_bin_index(angle):
    temp = (angle / 20.0) - 0.5
    j = math.floor(temp)
    return j

def calculate_center_of_bin(j):
    return round((j + 0.5) * 20.0, 9)

def calculate_value_of_bin(magnitude, angle, j):
    cj = calculate_center_of_bin(j)
    vj = magnitude * ((cj - angle) / 20.0)
    return round(vj, 9)

In [6]:
def extract_hog_features(image):
    resized_image = cv2.resize(image, (64, 128))

    hog = cv2.HOGDescriptor()
    features = hog.compute(resized_image)
    if features is None:
        print("Erro ao extrair características HOG: 'features' é None")
        return np.array([])
    # print(f"Dimensões das características HOG: {features.shape}")
    return np.array(features)
    # Gx = cv2.filter2D(resized_image, -1, np.array([[-1, 0, 1], 
    #            [-2, 0, 2], 
    #            [-1, 0, 1]]))
    # Gy = cv2.filter2D(resized_image, -1, np.array([[1, 2, 1],
    #            [0, 0, 0],
    #            [-1, -2, -1]]))
    
    # magnitude = np.sqrt(Gx**2 + Gy**2)
    # angle = np.arctan2(Gy, Gx) * (180 / np.pi)
    # angle[angle < 0] += 360
    
    # nrows, ncols = magnitude.shape
    # hog_features = []
    # num_bins = 9
    # bin_size = 180 // num_bins
        
    # for i in range(0, nrows, 8):
    #     temp = []
    #     for j in range(0, ncols, 8):
    #         block_magnitude = magnitude[i:i+8, j:j+8]
    #         block_angle = angle[i:i+8, j:j+8]
            
    #         for k in range(8):
    #             for l in range(8):
    #                 bins = [0.0] * num_bins
    #                 j = calculate_bin_index(block_angle[k, l])
    #                 vj = calculate_value_of_bin(block_magnitude[k, l], block_angle[k, l], j)
    #                 vj_1 = magnitude[k, l] - vj
    #                 bins[j] += vj
    #                 bins[j + 1] += vj_1
    #                 bins = [round(x, 9) for x in bins]
    #         temp.append(bins)
    #     hog_features.append(temp)
                
    # epsilon = 1e-5
    # feature_vectors = []
    # for i in range(0, len(hog_features) - 1, 1):
    #     temp = []
    #     for j in range(0, len(hog_features[0]) - 1, 1):
    #         values = [[hog_features[i][x] for x in range(j, j+2)] for i in range(i, i+2)]
    #         final_vector = []
    #         for k in values:
    #             for l in k:
    #                 for m in l:
    #                     final_vector.append(m)
    #         k = round(math.sqrt(sum([pow(x, 2) for x in final_vector])), 9)
    #         final_vector = [round(x/(k + epsilon), 9) for x in final_vector]
    #         temp.append(final_vector)
    #     feature_vectors.append(temp)
        
    # return [0.0 if x is None else x for x in feature_vectors]
    # Normalização L2
    # hog_features = np.array(hog_features)
    # norm = np.linalg.norm(hog_features) + 1e-7
    # hog_features = hog_features / norm
    # return hog_features.tolist()

In [ ]:
def create_features_df(folder):
    if os.path.exists(f'{folder}_features.csv'):
        print(f"Arquivo {folder}_features.csv já existe. Carregando...")
        return pd.read_csv(f'{folder}_features.csv')
    
    print(f"Extraindo HOG features de {folder}...")
    features = []
    images = load_images_from_folder(folder)
    
    for char, char_images in tqdm(images.items(), desc="Processando caracteres"):
        print(f"Processando imagens do caractere: {char}")
        for img in char_images:
            hog_features = extract_hog_features(img)
            features.append([char] + hog_features.tolist())
            
    df = pd.DataFrame(features)
    df.columns = ['label'] + [f'feature_{i}' for i in range(len(df.columns) - 1)]
    df.to_csv(f'{folder}_features.csv', index=False)
    
    return df

: 

In [ ]:
df_train = create_features_df(train_dir)
df_val = create_features_df(val_dir)
df_test = create_features_df(test_dir)

# X_train = df_train.drop(columns=['label'])
# y_train = df_train['label']
# X_val = df_val.drop(columns=['label'])
# y_val = df_val['label']
# X_test = df_test.drop(columns=['label'])
# y_test = df_test['label']

# model = RandomForestClassifier(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)

# y_pred_train = model.predict(X_train)
# y_pred_val = model.predict(X_val)
# y_pred_test = model.predict(X_test)

# print("Acurácia - Treinamento:", model.score(X_train, y_train))
# print("Acurácia - Validação:", model.score(X_val, y_val))
# print("Acurácia - Teste:", model.score(X_test, y_test))

# print("Relatório de Classificação - Treinamento:")
# print(classification_report(y_train, y_pred_train))
# print("Relatório de Classificação - Validação:")
# print(classification_report(y_val, y_pred_val))
# print("Relatório de Classificação - Teste:")
# print(classification_report(y_test, y_pred_test))
# cm = confusion_matrix(y_test, y_pred_test)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
# disp.plot(cmap=plt.cm.Blues)
# plt.title("Matriz de Confusão - Teste")
# plt.show()

Extraindo HOG features de dados/processed/treinamento...


Processando caracteres:   0%|          | 0/36 [00:00<?, ?it/s]

Processando imagens do caractere: 0


Processando caracteres:   3%|▎         | 1/36 [00:00<00:17,  1.97it/s]

Processando imagens do caractere: 8


...

Processando imagens do caractere: 6


Processando caracteres:  69%|██████▉   | 25/36 [00:17<00:09,  1.17it/s]

Processando imagens do caractere: Y


Processando caracteres:  72%|███████▏  | 26/36 [00:18<00:08,  1.15it/s]

Processando imagens do caractere: 3
